In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.autograd import Variable

In [16]:
batch_size = 64

train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
# in_channel = depth of image, out_channel = depth of 
# activation maps (no of filters for different features)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320,10)
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) #flatten
        x = self.fc(x)
        return F.log_softmax(x)

In [18]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [19]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [20]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [21]:
for epoch in range(1, 10):
    train(epoch)
    test()

/tmp/ipykernel_40271/2101139191.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/home/ricky/miniconda3/lib/python3.9/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370151529/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298622
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.299631
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297434
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.289050
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.290050
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.271144
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.258283
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.253298
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.205621
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.201804
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.188606
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.140683
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.087896
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.912775
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.900768
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.612620
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.433654
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.404921
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.004208
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.891759
Train Epoch: 1 [12800/60000 (

/tmp/ipykernel_40271/2453309322.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/home/ricky/miniconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1949, Accuracy: 9429/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.156748
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.186875
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.149754
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.149086
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.343455
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.379511
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.131090
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.119547
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.194458
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.074161
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.197488
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.080376
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.241803
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.167660
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.073877
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.137754
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.163071
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.118902
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.116832
Train Epoch: 2 [12

Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.045403
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.036423
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.036716
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.120045
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.065190
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.021608
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.060075
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.087074
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.070768
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.047283
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.252810
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.073543
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.191905
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.084423
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.047810
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.114353
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.311369
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.062947
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.227284
Train Epoch: 3 [56960/60000 (95%)]	Loss: 0.202848


Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.036472
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.029549
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.053794
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.059013
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.072485
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.090170
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.124099
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.081807
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.043642
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.046923
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.066420
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.120684
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.084199
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.030690
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.091795
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.039091
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.073029
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.079572
Train Epoch: 5 [40960/60000 (68%)]	Loss: 0.098162
Train Epoch: 5 [41600/60000 (69%)]	Loss: 0.138726


Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.188866
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.103038
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.012782
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.065936
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.037652
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.029098
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.011229
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.053811
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.144336
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.026943
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.092202
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.070654
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.072523
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.064888
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.027791
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.059257
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.093579
Train Epoch: 7 [24960/60000 (42%)]	Loss: 0.073288
Train Epoch: 7 [25600/60000 (43%)]	Loss: 0.017766
Train Epoch: 7 [26240/60000 (44%)]	Loss: 0.023352


Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.048105
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.013529

Test set: Average loss: 0.0478, Accuracy: 9847/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.031521
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.052218
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.040204
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.160921
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.099571
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.011435
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.076944
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.032444
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.089769
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.005410
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.022950
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.075393
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.121549
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.033893
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.016146
Train Epoch: 9 [9600/60000 (16%)]	Loss: 0.064149
Train Epoch: 9 [10240/60000 (17%)]	Loss: 0.161060
Train Epoch: 9 [10